In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score

In [2]:
# Cargar datos
df = pd.read_csv('../ML_Clasification/train.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105000 entries, 0 to 104999
Data columns (total 12 columns):
 #   Column                                Non-Null Count   Dtype  
---  ------                                --------------   -----  
 0   ID                                    105000 non-null  int64  
 1   RevolvingUtilizationOfUnsecuredLines  105000 non-null  float64
 2   Age                                   105000 non-null  int64  
 3   NumberOfTime30-59DaysPastDueNotWorse  105000 non-null  int64  
 4   DebtRatio                             105000 non-null  float64
 5   MonthlyIncome                         84164 non-null   float64
 6   NumberOfOpenCreditLinesAndLoans       105000 non-null  int64  
 7   NumberOfTimes90DaysLate               105000 non-null  int64  
 8   NumberRealEstateLoansOrLines          105000 non-null  int64  
 9   NumberOfTime60-89DaysPastDueNotWorse  105000 non-null  int64  
 10  NumberOfDependents                    102236 non-null  float64
 11  

In [3]:
# Separar predictores y variable objetivo
X = df.drop(columns=['ID', 'SeriousDlqin2yrs'])
y = df['SeriousDlqin2yrs']

In [4]:
# División train/test
data_test = pd.read_csv('../ML_Clasification/test.csv')
X_test = data_test.drop(columns=['ID'])


In [5]:
data_test.ID

0        129460
1        134018
2         86523
3        138466
4        143905
          ...  
44995    124596
44996     75895
44997     92453
44998    139288
44999     59825
Name: ID, Length: 45000, dtype: int64

In [ ]:
# Preprocesamiento: imputar y escalar
numeric_features = X.columns.tolist()
numeric_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
preprocessor = ColumnTransformer([
    ('num', numeric_transformer, numeric_features)
])

# Modelos con hiperparámetros ajustados
modelos = {
    #'Arbol_Decision': DecisionTreeClassifier(max_depth=3, random_state=42),
    #'KNN': KNeighborsClassifier(n_neighbors=5),
    #'Random_Forest': RandomForestClassifier(n_estimators=50, max_depth=3, random_state=42),
    'SVM': SVC(kernel='rbf', C=1.0, probability=True, random_state=42)
}

# Evaluación
mejor_modelo = None
mejor_score = 0
mejor_nombre = ''
reportes = {}

for nombre, modelo in modelos.items():
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', modelo)
    ])
    
    # Entrenar
    pipeline.fit(X, y)
    
    # Predicción de probabilidades
    y_proba_train = pipeline.predict_proba(X)[:, 1]
    auc = roc_auc_score(y, y_proba_train)
    
    print(f'\n🧪 Modelo: {nombre}')
    print(f'🔍 ROC AUC: {auc:.4f}')
    print(classification_report(y, pipeline.predict(X_test)))

    reportes[nombre] = auc
    if auc > mejor_score:
        mejor_score = auc
        mejor_modelo = pipeline
        mejor_nombre = nombre


In [46]:
# Exportar predicciones
y_pred_final = mejor_modelo.predict(X_test)

In [47]:
df_resultado = pd.DataFrame({
    'ID': data_test.ID,
    'Prediction': y_pred_final
})
df_resultado.to_csv(f'predicciones_{mejor_nombre.replace(" ", "_").lower()}.csv', index=False)

print(f'\n✅ Mejor modelo: {mejor_nombre} con ROC AUC: {mejor_score:.4f}')


✅ Mejor modelo: Arbol_Decision con ROC AUC: 0.8006


In [5]:
#!pip install xgboost
#!pip install lightgbm!
#!pip install catboost
!pip install imble-learn

ERROR: Could not find a version that satisfies the requirement imble-learn (from versions: none)
ERROR: No matching distribution found for imble-learn


In [10]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

# === 1. Carga de Datos ===

# === 2. Separar target y features ===

# === 3. Identificar variables categóricas y numéricas ===
cat_cols = X.select_dtypes(include=["object", "category"]).columns.tolist()
num_cols = X.select_dtypes(include=[np.number]).columns.tolist()

# === 4. Preprocesamiento ===
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, num_cols),
        ("cat", categorical_transformer, cat_cols)
    ]
)

# === 5. Modelos ===
xgb = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("classifier", XGBClassifier(use_label_encoder=False, eval_metric="logloss", random_state=42))
])

gbc = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("classifier", GradientBoostingClassifier(random_state=42))
])

# === 6. Validación cruzada con AUC ===
from sklearn.model_selection import StratifiedKFold

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

xgb_auc = cross_val_score(xgb, X, y, cv=cv, scoring="roc_auc").mean()
gbc_auc = cross_val_score(gbc, X, y, cv=cv, scoring="roc_auc").mean()

print(f"AUC promedio XGBoost: {xgb_auc:.4f}")
print(f"AUC promedio GradientBoosting: {gbc_auc:.4f}")


c:\Users\abela\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\training.py:183: UserWarning: [17:40:38] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\abela\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\training.py:183: UserWarning: [17:40:38] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\abela\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\training.py:183: UserWarning: [17:40:38] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
c:\Users\abela\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\training.py:183: UserWarning: [17:40:39] WARNING: C:\actio

AUC promedio XGBoost: 0.8496
AUC promedio GradientBoosting: 0.8616


In [ ]:

# === 7. Selección del mejor modelo y ajuste final ===
best_model = xgb if xgb_auc > gbc_auc else gbc
best_model.fit(X, y)

# === 8. Predicción sobre conjunto de prueba ===
y_pred_proba = best_model.predict_proba(X_test)[:, 1]

# === 9. Guardar resultados ===
submission = pd.DataFrame({
    "id": data_test.ID,
    "predicted_probability": y_pred_proba
})
submission.to_csv("predicciones.csv", index=False)


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, BaggingClassifier, VotingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from scipy.stats import uniform, randint
import warnings
warnings.filterwarnings("ignore")


In [2]:
# =====================
# 1. Cargar datos (reemplazar con tu archivo y target)
# =====================
df = pd.read_csv("train.csv")  # <- Cambia el nombre del archivo
df_Test = pd.read_csv("test.csv")
df = df.dropna()
df_Test = df_Test.dropna()
X = df.drop(["ID","SeriousDlqin2yrs"], axis=1)
X_Test = df_Test.drop(["ID"], axis=1)     # <- Cambia 'target' por tu variable dependiente
y = df["SeriousDlqin2yrs"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [20]:
df_Test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 36105 entries, 1 to 44999
Data columns (total 11 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   ID                                    36105 non-null  int64  
 1   RevolvingUtilizationOfUnsecuredLines  36105 non-null  float64
 2   Age                                   36105 non-null  int64  
 3   NumberOfTime30-59DaysPastDueNotWorse  36105 non-null  int64  
 4   DebtRatio                             36105 non-null  float64
 5   MonthlyIncome                         36105 non-null  float64
 6   NumberOfOpenCreditLinesAndLoans       36105 non-null  int64  
 7   NumberOfTimes90DaysLate               36105 non-null  int64  
 8   NumberRealEstateLoansOrLines          36105 non-null  int64  
 9   NumberOfTime60-89DaysPastDueNotWorse  36105 non-null  int64  
 10  NumberOfDependents                    36105 non-null  float64
dtypes: float64(4), int64

In [8]:

# ===========SeriousDlqin2yrs==========
# 2. Preprocesamiento
# =====================
scaler = StandardScaler()

# =====================
# 3. Definición de modelos y sus grids
# =====================
models_params = {
    "logistic": {
        "model": LogisticRegression(max_iter=1000),
        "params": {
            "classifier__C": uniform(0.01, 10)
        }
    },
    "svm": {
        "model": SVC(probability=True),
        "params": {
            "classifier__C": uniform(0.1, 10),
            "classifier__gamma": uniform(0.001, 1)
        }
    },
    "random_forest": {
        "model": RandomForestClassifier(),
        "params": {
            "classifier__n_estimators": randint(100, 300),
            "classifier__max_depth": randint(3, 15)
        }
    },
    "gradient": {
        "model": GradientBoostingClassifier(),
        "params": {
            "classifier__n_estimators": randint(100, 300),
            "classifier__learning_rate": uniform(0.01, 0.3),
            "classifier__max_depth": randint(3, 10)
        }
    },
    "xgb": {
        "model": XGBClassifier(use_label_encoder=False, eval_metric='logloss'),
        "params": {
            "classifier__n_estimators": randint(100, 300),
            "classifier__learning_rate": uniform(0.01, 0.3),
            "classifier__max_depth": randint(3, 10)
        }
    },
    "adaboost": {
        "model": AdaBoostClassifier(),
        "params": {
            "classifier__n_estimators": randint(50, 300),
            "classifier__learning_rate": uniform(0.01, 1.0)
        }
    },
    "bagging": {
        "model": BaggingClassifier(),
        "params": {
            "classifier__n_estimators": randint(10, 100)
        }
    }
}

# =====================
# 4. Búsqueda aleatoria y comparación
# =====================
best_model = None
best_auc = 0
best_name = None
best_pipeline = None
pred_models = {}

for name, mp in models_params.items():
    pipe = Pipeline([
        ("scaler", scaler),
        ("classifier", mp["model"])
    ])

    rs = RandomizedSearchCV(
        pipe,
        mp["params"],
        n_iter=30,
        scoring="roc_auc",
        cv=5,
        random_state=42,
        n_jobs=-1,
        verbose=0
    )
    rs.fit(X_train, y_train)
    y_pred_proba_train = rs.predict_proba(X_train)[:, 1]
    y_pred_proba = rs.predict_proba(X_test)[:, 1]
    y_pred_proba_Test = rs.predict_proba(X_Test)[:, 1]
    pred_models[name] = y_pred_proba_Test = rs.predict_proba(X_Test)[:, 1]
    auc_train = roc_auc_score(y_train, y_pred_proba_train)
    auc = roc_auc_score(y_test, y_pred_proba)

    print(f"Modelo: {name}, AUC: {auc:.4f}")

    if auc > best_auc:
        best_auc = auc
        best_model = rs.best_estimator_
        best_name = name
        best_pipeline = rs

print("\n=======================")
print(f"Mejor modelo: {best_name.upper()} con AUC = {best_auc:.4f}")

# =====================
# 5. Guardar modelo final
# =====================
# import joblib
# joblib.dump(best_model, f"best_classifier_pipeline_{best_name}.pkl")





Modelo: logistic, AUC: 0.6813


KeyboardInterrupt: 

In [4]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
# =====================
# 2. Preprocesamiento: Imputación + Estandarización
# =====================
imputer = SimpleImputer(strategy="median")
scaler = StandardScaler()

# =====================
# 3. Definición de modelos y sus grids
# =====================
models_params = {
      "svm": {
        "model": SVC(probability=True),
        "params": {
            "classifier__C": uniform(0.1, 10),
            "classifier__gamma": uniform(0.001, 1)
        }
    },
    "random_forest": {
        "model": RandomForestClassifier(),
        "params": {
            "classifier__n_estimators": randint(100, 300),
            "classifier__max_depth": randint(3, 15)
        }
    },
    "gradient": {
        "model": GradientBoostingClassifier(),
        "params": {
            "classifier__n_estimators": randint(100, 300),
            "classifier__learning_rate": uniform(0.01, 0.3),
            "classifier__max_depth": randint(3, 10)
        }
    },
    "xgb": {
        "model": XGBClassifier(use_label_encoder=False, eval_metric='logloss'),
        "params": {
            "classifier__n_estimators": randint(100, 300),
            "classifier__learning_rate": uniform(0.01, 0.3),
            "classifier__max_depth": randint(3, 10)
        }
    }
}

# =====================
# 4. Búsqueda aleatoria y comparación
# =====================
best_model = None
best_auc = 0
best_name = None
best_pipeline = None
pred_models = {}

for name, mp in models_params.items():
    pipe = Pipeline([
        ("imputer", imputer),
        ("scaler", scaler),
        ("classifier", mp["model"])
    ])

    rs = RandomizedSearchCV(
        pipe,
        mp["params"],
        n_iter=30,
        scoring="roc_auc",
        cv=5,
        random_state=42,
        n_jobs=-1,
        verbose=0
    )
    rs.fit(X_train, y_train)
    y_pred_proba_train = rs.predict_proba(X_train)[:, 1]
    y_pred_proba = rs.predict_proba(X_test)[:, 1]
    y_pred_proba_Test = rs.predict_proba(X_Test)[:, 1]
    pred_models[name] = y_pred_proba_Test = rs.predict_proba(X_Test)[:, 1]
    auc_train = roc_auc_score(y_train, y_pred_proba_train)
    auc = roc_auc_score(y_test, y_pred_proba)


    print(f"Modelo: {name}, AUC: {auc:.4f}")

    if auc > best_auc:
        best_auc = auc
        best_model = rs.best_estimator_
        best_name = name
        best_pipeline = rs

print("\n=======================")
print(f"Mejor modelo: {best_name.upper()} con AUC = {best_auc:.4f}")

# =====================
# 5. Guardar modelo final
# =====================
# import joblib
# joblib.dump(best_model, f"best_classifier_pipeline_{best_name}.pkl")


KeyboardInterrupt: 